# Customer's error function

To the error function it is expected the following pandas.DataFrame struncture:

| StoreID  | Month | Region | NumberOfSales | _NumberOfSales |
| ---------|----------------|---------------| ---------------|
| 1000	   |3      | 4      | 16            | 16             |
| 1000	   |4      | 4      | 30            | 23             |
| 1001	   |3      | 6      | 410           | 411            |
| 1001	   |4      | 27     | 3130          | 3120           |
| 1002	   |3      | 58     | 10            | 8              |

Where:
 
 - *NumberOfSales* are the test values
 - *_NumberOfSales* are the predicted values
 
 

Start from test set to simulate a predicted dataset

In [1]:
from import_man import *

error_evaluation_columns = ['StoreID', 'Month', 'Region', 'NumberOfSales', '_NumberOfSales']

df = pd.read_csv('./dataset/train.csv')

print("Shape before: " + str(df.shape))

# Let's work on a reduced instance of the test set
df = df.sample(n=5000)

print("Shape after: " + str(df.shape))

Shape before: (523021, 36)
Shape after: (5000, 36)


In [2]:
from datetime import datetime

# Parse the month and add it as attribute
df['Month'] =  df['Date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').month)
df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees,Month
244104,1348,13/02/2017,0,1,0,Standard Market,With Non-Food Department,13105,6,165,...,12.0,6,-4,50,991,-3,10.0,1.02,86,2
347896,1497,30/12/2016,0,1,0,Hyper Market,With Non-Food Department,2372,2,319,...,10.0,14,-2,66,1007,-2,8.0,0.00,168,12
70864,1101,31/08/2017,0,1,0,Hyper Market,General,27399,5,151,...,21.0,5,6,21,1015,7,16.0,0.00,-1,8
164837,1235,29/07/2017,0,1,0,Hyper Market,General,2080,3,158,...,11.0,6,12,26,1012,17,1.0,0.00,68,7
361288,1518,13/04/2016,0,1,1,Hyper Market,With Non-Food Department,6593,10,231,...,4.0,14,0,81,1005,3,2.0,0.00,104,4


In [3]:
# Create fake predicted sales 
df['_NumberOfSales'] = df.NumberOfSales.apply(lambda x: df['NumberOfSales'].sample().values[0])

# Remove useless columns
df =  df[error_evaluation_columns]

df.head(20)

,StoreID,Month,Region,NumberOfSales,_NumberOfSales
244104,1348,2,6,3375,3968
347896,1497,12,2,4975,2152
70864,1101,8,5,2217,4797
164837,1235,7,3,1968,0
361288,1518,4,10,3814,0
83178,1119,6,2,4438,4491
75452,1108,4,10,5960,12506
54252,1077,1,9,10077,2804
217366,1310,3,9,4095,2672
147917,1211,2,6,4659,5494


In [8]:
df_sums_by_region = df.groupby(['Region']).sum()

df_sums_by_region['abs_a_minus_p'] = abs(df_sums_by_region['NumberOfSales'] - df_sums_by_region['_NumberOfSales'])

df_sums_by_region.head(20)

,StoreID,Month,NumberOfSales,_NumberOfSales,abs_a_minus_p
Region,,,,,
0,548243,2514,1623894,1568527,55367
1,225576,1162,644634,639409,5225
2,852281,3503,2381934,2576616,194682
3,906960,4190,3206405,2747542,458863
4,200278,935,693130,595349,97781
5,468691,2258,1325539,1537400,211861
6,339857,1665,904443,958706,54263
7,549361,2697,1452746,1572095,119349
8,230969,995,533578,672404,138826


In [11]:
# Total error per region
df_sums_by_region['E_r'] = df_sums_by_region['abs_a_minus_p'] / df_sums_by_region['NumberOfSales']

df_sums_by_region.head(20)

,StoreID,Month,NumberOfSales,_NumberOfSales,abs_a_minus_p,E_r
Region,,,,,,
0,548243,2514,1623894,1568527,55367,0.034095
1,225576,1162,644634,639409,5225,0.008105
2,852281,3503,2381934,2576616,194682,0.081733
3,906960,4190,3206405,2747542,458863,0.143108
4,200278,935,693130,595349,97781,0.141072
5,468691,2258,1325539,1537400,211861,0.159830
6,339857,1665,904443,958706,54263,0.059996
7,549361,2697,1452746,1572095,119349,0.082154
8,230969,995,533578,672404,138826,0.260179


In [12]:
# Get the number of regions
N_regions = len(df.Region.unique())

print("Number of regions: {}".format(N_regions))

df_err = df_ssr[df_ssr.index < 2]
df_err.head()

AttributeError: 'DataFrame' object has no attribute 'Region'

In [7]:
Error = df_err['E_r'].sum() / N_regions

print("Total error: {}".format(Error))

Total error: 0.0038364163858320937
